In [ ]:
while(True):
    import requests
    from bs4 import BeautifulSoup
    from datetime import datetime,timedelta
    import pandas as pd
    import numpy as np
    saat=datetime.now().hour
    import cloudscraper
    from datetime import datetime
    from concurrent.futures import ThreadPoolExecutor,as_completed
    simdi = datetime.now()
    saat1 = simdi.hour
    dakika = simdi.minute
    import time

    if (saat == 23 and dakika >= 30) or (saat == 0 and dakika == 0):
        
        hedef_zaman = datetime(simdi.year, simdi.month, simdi.day, 0, 0) + timedelta(days=1 if saat == 0 else 0)
        while datetime.now() < hedef_zaman:
            print("23:30 ile 00:00 arasında döngü başlamaz. Bekleniyor...")
            time.sleep(60)  
        continue  
    
    # Döngü çalışabilir durumda
    print("Döngü başladı, saat:", simdi.strftime("%H:%M:%S"))

                

    bugün=datetime.now().strftime("%Y-%m-%d")



    dün=(datetime.now()-timedelta(days=1)).strftime("%Y-%m-%d")


    data=pd.read_csv("sepet.csv")
    try:
        data=data.set_index(data["Unnamed: 0"]).drop("Unnamed: 0",axis=1)
    except:
        data=data.set_index(data["original_index"]).drop("original_index",axis=1)
    data.index.name=""





    def veriekle(ürün, data, urunler_df):
        if urunler_df is None or urunler_df.empty:
            return data
        else:

            if isinstance(data.loc[ürün], pd.Series):
                    urun_data=pd.DataFrame(data.loc[ürün]).T
                    urun_data = pd.merge(
                        urun_data, 
                        urunler_df, 
                        on='Ürün', 
                        how='outer'
                    )
                
                # Çakışan sütunları birleştir
                    if f'{bugün}_x' in urun_data.columns and f'{bugün}_x' in urun_data.columns:
                        urun_data[f'{bugün}'] = urun_data[f'{bugün}_x'].combine_first(urun_data[f'{bugün}_y'])
                        urun_data.drop([f'{bugün}_x', f'{bugün}_y'], axis=1, inplace=True)
                    
                    # İndeksi düzenle
                        urun_data = urun_data.set_index([[ürün] * len(urun_data)])
                        
                        # Mevcut ürünü veri setinden kaldır
                        data = data.drop(ürün, axis=0)
                        
                        # Yeni veriyi ekle
                        data = pd.concat([data, urun_data], axis=0)
                
                
            else:
                
                urun_data = data.loc[ürün]
                
                # Merge işlemi sırasında sütun çakışması yaşanırsa, önekler oluşur
                urun_data = pd.merge(
                    urun_data, 
                    urunler_df, 
                    on='Ürün', 
                    how='outer'
                )
                
                # Çakışan sütunları birleştir
                if f'{bugün}_x' in urun_data.columns and f'{bugün}_x' in urun_data.columns:
                    urun_data[f'{bugün}'] = urun_data[f'{bugün}_x'].combine_first(urun_data[f'{bugün}_y'])
                    urun_data.drop([f'{bugün}_x', f'{bugün}_y'], axis=1, inplace=True)
                
                # İndeksi düzenle
                urun_data = urun_data.set_index([[ürün] * len(urun_data)])
                
                # Mevcut ürünü veri setinden kaldır
                data = data.drop(ürün, axis=0)
                
                # Yeni veriyi ekle
                data = pd.concat([data, urun_data], axis=0)

            return data
                
                







    import re
    import requests
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from time import sleep
    import pandas as pd
    from webdriver_manager.chrome import ChromeDriverManager

    options=Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    def clean_price(price_text):
        # Remove any non-numeric characters except for commas and dots
        price_text = re.sub(r'[^\d,.]', '', price_text)
        # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
        price_text = price_text.replace(',', '.')
        try:
            return float(price_text)
        except ValueError:
            return None



    import re
    import requests
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from time import sleep
    import pandas as pd
    from webdriver_manager.chrome import ChromeDriverManager




    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # Function to clean and convert price text to float
    def clean_price(price_text):
        # Remove any non-numeric characters except for commas and dots
        price_text = re.sub(r'[^\d,.]', '', price_text)
        # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
        price_text = price_text.replace(',', '.')
        try:
            return float(price_text)
        except ValueError:
            return None

    # Function to scrape product names and prices from Migros for multiple URLs
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from webdriver_manager.chrome import ChromeDriverManager
    import re


    def vericek(carrefour,migros,name):
        def scrape_migros_products(pages):
            
                try:
                    urunad=[]
                    urunfiyat=[]


                    headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
                            "Accept":"application/json, text/plain, */*"}



                    temizlinkler=pages
                    scraper=cloudscraper.CloudScraper()
                    def sayfasayısı(url):
                        scraper=cloudscraper.CloudScraper()
                        return scraper.get(url,headers=headers).json()["data"]["searchInfo"]["pageCount"]

                    def sayfa_verisini_cek(url,sayfa):
                        scraper=cloudscraper.CloudScraper()
                        response=scraper.get(f"{url}?page={sayfa}",headers=headers).json()
                        return response["data"]["searchInfo"]["storeProductInfos"]

                    with ThreadPoolExecutor(max_workers=5) as executor:
                        futures=[]
                        for i in temizlinkler:
                            sayfa_sayisi=sayfasayısı(i)
                            for j in range(1,sayfa_sayisi+1):
                                futures.append(executor.submit(sayfa_verisini_cek,i,j))
                        
                        for future in as_completed(futures):
                            info=future.result()
                            for urun in info:
                                urunad.append(urun.get("name"))
                                urunfiyat.append(int(urun.get("salePrice"))/100)
                    

                    
                

                    veri=pd.DataFrame({"Product Name":urunad,"Price (TRY)":urunfiyat})
                    return veri
                except:
                    print("Tekrar deneniyor...")
                    time.sleep(5)






            

            
        def scrape_carrefour_products(pages):
            carrefour_data = []
            
            for url in pages:
                print(f"Scraping URL: {url}")
                driver.get(url)

                try:
                    # Find all product links
                    product_elements = driver.find_elements(By.CSS_SELECTOR, 'a.product-return')

                    for index, product_element in enumerate(product_elements):
                        try:
                            # Extract product name
                            product_name_element = product_element.find_element(By.CSS_SELECTOR, 'h3.item-name')
                            product_name = product_name_element.text.strip()

                            # Extract product price
                            # First, try to find the "priceLineThrough" element for non-discounted price
                            try:
                                price_line_through_element = product_element.find_element(By.CSS_SELECTOR, 'span.priceLineThrough.js-variant-price')
                                price_line_through_text = price_line_through_element.text.strip()
                                product_price = clean_price(price_line_through_text)
                            except:
                                # If no "priceLineThrough", fall back to the current price
                                price_element = product_element.find_element(By.CSS_SELECTOR, 'span.item-price.js-variant-discounted-price')
                                product_price_text = price_element.text.strip()
                                product_price = clean_price(product_price_text)

                            if product_name and product_price is not None:
                                carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                                print(f"Collected - Product: {product_name}, Price: {product_price} TRY")
                            else:
                                print(f"Skipping product with incomplete data on {url}")

                        except Exception as e:
                            print(f"Error extracting data from product {index + 1} on {url}: {e}")
                    
                    sleep(2)  # Sleep to mimic human interaction and avoid detection
                
                except Exception as e:
                    print(f"Error while scraping {url}: {e}")

            return carrefour_data

        


        migros_data = scrape_migros_products(migros)

        try:
            if carrefour=="":
                carrefour_data=None
                all_data=None
            
                
            

            elif carrefour!="":
                carrefour_data = scrape_carrefour_products(carrefour)
                carrefour_data=pd.DataFrame(carrefour_data)
                all_data=pd.concat([migros_data,carrefour_data],axis=0)
                if carrefour_data.empty or carrefour_data is None:
                    all_data =  migros_data

            if migros_data is None or migros_data.empty:
                all_data =   carrefour_data
                
            
            if not migros_data is None:
                if not carrefour_data is None:
                    if not carrefour_data.empty:
                        if not migros_data.empty:
                            all_data=pd.concat([migros_data,carrefour_data],axis=0)
        except:
            all_data=pd.concat([migros_data,carrefour_data],axis=0)
        if not all_data is None:
            if not all_data.empty:
                if len(all_data)>1:
                    product_df = pd.DataFrame(all_data)
                    product_df=product_df.reset_index()

                    product_df['Product Group'] = product_df['Product Name'].str.lower()

                    # Geometrik ortalama hesaplama fonksiyonu
                    def geometric_mean(prices):
                        return np.exp(np.mean(np.log(prices)))

                    geo_mean_df = (
                        product_df.groupby(['Product Group'])
                        .agg(
                            Geometric_Mean=('Price (TRY)', geometric_mean),
                            Product_Names=('Product Name', lambda x: ', '.join(x))
                        )
                        .reset_index()
                    )
                    product_df=geo_mean_df.copy()
                    product_df=product_df.drop("Product Group",axis=1)
                    product_df.columns=["Price (TRY)","Product Name"]

                    urunler_df = product_df.copy()
                    if urunler_df is not None and not urunler_df.empty:
                            urunler_df.columns=[str(bugün),"Ürün"]
                        

                            urunler_df.index=len(urunler_df)*[name]
                            urunler_df=urunler_df.drop_duplicates()
                            urunler_df=urunler_df.dropna()

                            return urunler_df

        try:
            all_data.columns=["Ürün",str(bugün)]
            return all_data
        except:
            return pd.DataFrame()

    carrefour = ["https://www.carrefoursa.com/pirinc/c/1134?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]
    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/pirinc-c-427"
    ]



    urunler_df=vericek(carrefour,migros_pages,"Pirinç")


    data=veriekle("Pirinç",data,urunler_df)
        






    pages1 = ["https://www.carrefoursa.com/bugday-unu/c/1277?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]

        



    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/sade-un-c-289b"]

    #
    urunler_df=vericek(pages1,migros_pages,"Buğday Unu")






    data=veriekle("Buğday Unu",data,urunler_df)









    pages1=["https://www.carrefoursa.com/search?q=devam+s%C3%BCt%C3%BC%3AbestSeller%3AproductPrimaryCategoryCode%3A1848&show=All"]
        




    migros_pages =["https://www.migros.com.tr/rest/search/screens/devam-sutu-c-1136b"]


    urunler_df=vericek(pages1,migros_pages,"Bebek Sütü (Toz Karışım)")



    data=veriekle("Bebek Sütü (Toz Karışım)",data,urunler_df)








    pages1 = ["https://www.carrefoursa.com/bulgur/c/1142?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]
        




    # Define base URL and total pages to scrape for Migros
    migros_pages1= ["https://www.migros.com.tr/rest/search/screens/pilavlik-bulgur-c-2785"]



    urunler_df=vericek(pages1,migros_pages1,"Bulgur")

    data=veriekle("Bulgur",data,urunler_df)





    pages1 = ["https://www.migros.com.tr/rest/search/screens/ekmek-c-455"]
    carrefour=["https://www.carrefoursa.com/search?q=ekmek%3AbestSeller%3AinStockFlag%3Atrue%3AproductPrimaryCategoryCode%3A1401&show=All"]
        




    urunler_df=vericek(carrefour,pages1,"Ekmek")



    data=veriekle("Ekmek",data,urunler_df)






    pages1 = ["https://www.carrefoursa.com/biskuvi/c/1529?q=%3AbestSeller%3Acategory%3A1534%3Acategory%3A1533%3Acategory%3A1537%3Acategory%3A1530%3Acategory%3A1531%3Acategory%3A1536%3AinStockFlag%3Atrue&show=All"]

        
    migros=["https://www.migros.com.tr/rest/search/screens/biskuvi-c-43c"]

    urunler_df=vericek(pages1,migros,"Bisküvi")




    data=veriekle("Bisküvi",data,urunler_df)





    pages1= ["https://www.carrefoursa.com/search?q=kraker%3AbestSeller%3AinStockFlag%3Atrue&show=All"]









    # Define the list of pages to scrape from Migros
    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/kraker-c-27ea"]



    urunler_df=vericek(pages1,migros_pages,"Kraker")







    data=veriekle("Kraker",data,urunler_df)





    pages1= ["https://www.carrefoursa.com/search?q=gofret%3AbestSeller%3AinStockFlag%3Atrue&show=All"]

    migros=["https://www.migros.com.tr/rest/search/screens/gofret-c-43a"]
    urunler_df=vericek(pages1,migros,"Gofret")


    data=veriekle("Gofret",data,urunler_df)









    pages1= ["https://www.carrefoursa.com/pastalar/c/1289?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]




    # Define the list of pages to scrape from Migros
    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/pasta-c-459"]

    urunler_df=vericek(pages1,migros_pages,"Pasta")


    data=veriekle("Pasta",data,urunler_df)








    pages1= ["https://www.carrefoursa.com/search?q=kek%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/kek-c-43d"]

    urunler_df=vericek(pages1,migros_pages,"Kek")


    data=veriekle("Kek",data,urunler_df)









    pages1= ["https://www.carrefoursa.com/search?q=baklava%3AbestSeller%3AinStockFlag%3Atrue%3AproductPrimaryCategoryCode%3A1294"]




    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/pastane-tatlilar-c-287b"
    ]

    urunler_df=vericek(pages1,migros_pages,"Baklava")
    if urunler_df is not None and not urunler_df.empty:
        urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("Baklava|baklava|Baklavası|baklavası", regex=True)]

        data=veriekle("Baklava",data,urunler_df)




    # Define the list of pages to scrape from Migros
    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/yufka-c-2872"]

    urunler_df=vericek("",migros_pages,"Ekmek Hamuru (Yufka)")



    data=veriekle("Ekmek Hamuru (Yufka)",data,urunler_df)








    pages1= ["https://www.carrefoursa.com/makarna/c/1122?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    # Define the list of pages to scrape from Migros
    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/normal-makarna-c-2780"]

    urunler_df=vericek(pages1,migros_pages,"Makarna")


    data=veriekle("Makarna",data,urunler_df)





    pages1= ["https://www.carrefoursa.com/search/?q=%C5%9Fehriye%3Arelevance%3AinStockFlag%3Atrue&text=%C5%9Fehriye#"]



    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/normal-makarna-c-2780"]

    urunler_df=vericek(pages1,migros_pages,"Şehriye")


    data=veriekle("Şehriye",data,urunler_df)







    pages1= ["https://www.carrefoursa.com/musli-hububat-urunleri/c/1378?q=%3AbestSeller%3Acategory%3A1310%3AinStockFlag%3Atrue&show=All"]



        
        


    # Define the list of pages to scrape from Migros
    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/misir-gevregi-c-2773"]

    urunler_df=vericek(pages1,migros_pages,"Müsli ve Hububat Ürünleri")


    data=veriekle("Tahıl Gevreği",data,urunler_df)








    pages1= ["https://www.carrefoursa.com/search?q=dana+eti%3AbestSeller%3AinStockFlag%3Atrue%3AproductPrimaryCategoryCode%3A1046"]




    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/dana-eti-c-3fa"

    ]

    urunler_df=vericek(pages1,migros_pages,"Dana Eti")

    data=veriekle("Dana Eti",data,urunler_df)








    pages1= ["https://www.carrefoursa.com/kuzu/c/1054?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/kuzu-eti-c-3fb"]


    urunler_df=vericek(pages1,migros_pages,"Kuzu Eti")

    data=veriekle("Kuzu Eti",data,urunler_df)









    pages1= ["https://www.carrefoursa.com/pilic/c/1061?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]




    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/pilic-c-3fe"]

    urunler_df=vericek(pages1,migros_pages,"Tavuk Eti")




    data=veriekle("Tavuk Eti",data,urunler_df)


    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/sakatat-c-3fd"]

    urunler_df=vericek("",migros_pages,"Sakatat")


    data=veriekle("Sakatat",data,urunler_df)








    pages1= ["https://www.carrefoursa.com/sucuk/c/1077?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/sucuk-c-404"]



    urunler_df=vericek(pages1,migros_pages,"Sucuk")

    data=veriekle("Sucuk",data,urunler_df)





        
    pages1= ["https://www.carrefoursa.com/sosis/c/1084?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]


    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/sosis-c-405"]



    urunler_df=vericek(pages1,migros_pages,"Sosis")




    data=veriekle("Sosis",data,urunler_df)








        
    pages1= ["https://www.carrefoursa.com/salam-jambon-ve-fume/c/1092?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/salam-c-112d6"]

    urunler_df=vericek(pages1,migros_pages,"Salam")



    data=veriekle("Salam",data,urunler_df)



    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/pratik-yemek-c-44f?sayfa=1&90=503"]



    urunler_df=vericek("",migros_pages,"Hazır Et Yemekleri")


    data=veriekle("Hazır Et Yemekleri",data,urunler_df)







        
    pages1= ["https://www.carrefoursa.com/baliklar/c/1099?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/mevsim-baliklari-c-402"]


    urunler_df=vericek(pages1,migros_pages,"Balık")

    data=veriekle("Balık",data,urunler_df)


        
    pages1= ["https://www.carrefoursa.com/paketli-urunler/c/1068?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/konserve-deniz-urunleri-c-285a"]




    urunler_df=vericek(pages1,migros_pages,"Deniz Ürünleri")


    data=veriekle("Konserve Balık",data,urunler_df)







        
    pages1= ["https://www.carrefoursa.com/sut/c/1311?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]




    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/sut-c-6c"]



    urunler_df=vericek(pages1,migros_pages,"Süt")


    data=veriekle("Süt",data,urunler_df)







        
    pages1= ["https://www.carrefoursa.com/yogurt/c/1389?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/yogurt-c-6e"]



    urunler_df=vericek(pages1,migros_pages,"Yoğurt")



    data=veriekle("Yoğurt",data,urunler_df)








        
    pages1= ["https://www.carrefoursa.com/sutlu-tatli-puding/c/1962?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]




    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/sutlu-tatlilar-c-41c"]

    urunler_df=vericek(pages1,migros_pages,"Hazır Sütlü Tatlılar")





    data=veriekle("Hazır Sütlü Tatlılar",data,urunler_df)







        
    pages1= ["https://www.carrefoursa.com/beyaz-peynir/c/1319?q=%3AbestSeller&show=All"]



        

    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/beyaz-peynir-c-40b"]



    urunler_df=vericek(pages1,migros_pages,"Beyaz Peynir")





    data=veriekle("Beyaz Peynir",data,urunler_df)








        
    pages1= ["https://www.carrefoursa.com/kasar-/c/1324?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    # Define the list of pages to scrape from Migros
    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/kasar-peyniri-c-40d"]

    urunler_df=vericek(pages1,migros_pages,"Kaşar Peyniri")



    data=veriekle("Kaşar Peyniri",data,urunler_df)



    pages1= ["https://www.carrefoursa.com/krem-peynir/c/1336?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]



    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/krem-peynir-c-2737"]




    urunler_df=vericek(pages1,migros_pages,"Krem Peynir")



    data=veriekle("Krem Peynir",data,urunler_df)








    pages1= ["https://www.carrefoursa.com/yumurta/c/1349?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    migros_pages  =   ["https://www.migros.com.tr/rest/search/screens/yumurta-c-70"]



    urunler_df=vericek(pages1,migros_pages,"Yumurta")




    data=veriekle("Yumurta",data,urunler_df)







    pages1= ["https://www.carrefoursa.com/tereyag/c/1350?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/tereyagi-c-413"]



    urunler_df=vericek(pages1,migros_pages,"Tereyağı (Kahvaltılık)")


    data=veriekle("Tereyağı (Kahvaltılık)",data,urunler_df)






        
    pages1= ["https://www.carrefoursa.com/margarin/c/1351?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]




    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/paket-margarin-c-274e"]




    urunler_df=vericek(pages1,migros_pages,"Margarin")


    data=veriekle("Margarin",data,urunler_df)







        
    pages1= ["https://www.carrefoursa.com/zeytinyagi/c/1114?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]




    migros_pages  =  ["https://www.migros.com.tr/rest/search/screens/zeytinyagi-c-433"]

    urunler_df=vericek(pages1,migros_pages,"Zeytinyağı")

    data=veriekle("Zeytinyağı",data,urunler_df)







    pages1= ["https://www.carrefoursa.com/aycicek/c/1112?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]



    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/aycicek-yagi-c-42d"]



    urunler_df=vericek(pages1,migros_pages,"Ayçiçek Yağı")



    data=veriekle("Ayçiçek Yağı",data,urunler_df)








        
    pages1= ["https://www.carrefoursa.com/search?q=portakal%3AbestSeller%3AproductPrimaryCategoryCode%3A1016%3AinStockFlag%3Atrue&text=portakal#"]



    migros_pages  = ["https://www.migros.com.tr/rest/search/screens/narenciye-c-3ec?sayfa=1&93=652"]



    urunler_df=vericek(pages1,migros_pages,"Portakal")




    data=veriekle("Portakal",data,urunler_df)




    def vericek1(carrefour="",migros="",name=""):
        
        try:

    # Function to clean and convert price text to float
            def clean_price(price_text):
                # Remove any non-numeric characters except for commas and dots
                price_text = re.sub(r'[^\d,.]', '', price_text)
                # Replace commas with dots if needed (ensure it works with Turkish formatted numbers)
                price_text = price_text.replace(',', '.')
                try:
                    return float(price_text)
                except ValueError:
                    return None



            # Function to scrape product names and prices from Carrefour for multiple URLs
            def scrape_carrefour_products(pages):
                carrefour_data = []
                
                for url in pages:
                    print(f"Scraping URL: {url}")
                    driver.get(url)

                    try:
                        # Find all product links
                        product_elements = driver.find_elements(By.CSS_SELECTOR, 'a.product-return')

                        for index, product_element in enumerate(product_elements):
                            try:
                                # Extract product name
                                product_name_element = product_element.find_element(By.CSS_SELECTOR, 'h3.item-name')
                                product_name = product_name_element.text.strip()

                                # Extract product price
                                # First, try to find the "priceLineThrough" element for non-discounted price
                                try:
                                    price_line_through_element = product_element.find_element(By.CSS_SELECTOR, 'span.priceLineThrough.js-variant-price')
                                    price_line_through_text = price_line_through_element.text.strip()
                                    product_price = clean_price(price_line_through_text)
                                except:
                                    # If no "priceLineThrough", fall back to the current price
                                    price_element = product_element.find_element(By.CSS_SELECTOR, 'span.item-price.js-variant-discounted-price')
                                    product_price_text = price_element.text.strip()
                                    product_price = clean_price(product_price_text)

                                if product_name and product_price is not None:
                                    carrefour_data.append({"Product Name": product_name, "Price (TRY)": product_price})
                                    print(f"Collected - Product: {product_name}, Price: {product_price} TRY")
                                else:
                                    print(f"Skipping product with incomplete data on {url}")

                            except Exception as e:
                                print(f"Error extracting data from product {index + 1} on {url}: {e}")
                        
                        sleep(2)  # Sleep to mimic human interaction and avoid detection
                    
                    except Exception as e:
                        print(f"Error while scraping {url}: {e}")

                return carrefour_data


            # Define the list of pages to scrape from Migros
            migros_pages  = migros







            if migros:
                
                
                migros_data = scrape_migros_products(migros)

                
            if carrefour:
                carrefour_data = scrape_carrefour_products(carrefour)

            if migros_data and carrefour:

                all_data = migros_data + carrefour_data
            elif migros_data and not carrefour:
                all_data=migros_data
            else:
                all_data=carrefour_data
            product_df = pd.DataFrame(all_data)

            





            urunler_df = product_df.copy()
            if urunler_df is not None and not urunler_df.empty:
                urunler_df.columns=["Ürün",str(bugün)]
                urunler_df=urunler_df.groupby("Ürün", as_index=False).agg({str(bugün): 'mean'})

                urunler_df.index=len(urunler_df)*[name]
                urunler_df=urunler_df.drop_duplicates()
                urunler_df=urunler_df.dropna()

                return urunler_df
        except:
                pass





            

        





    carrefour=["https://www.carrefoursa.com/search?q=%C3%BCz%C3%BCm%3AbestSeller%3AproductPrimaryCategoryCode%3A1017%3AinStockFlag%3Atrue&text=%C3%BCz%C3%BCm#"]
    migros=["https://www.migros.com.tr/rest/search/screens/yumusak-meyveler-c-3ee"]

    urunler_df=vericek(carrefour,migros,"Üzüm")
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Üzüm|üzüm", regex=True)]
            data=veriekle("Üzüm",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=armut%3AbestSeller%3AproductPrimaryCategoryCode%3A1018%3AinStockFlag%3Atrue&text=armut#"]
    migros=["https://www.migros.com.tr/rest/search/screens/sert-meyveler-c-3ed?sayfa=1&93=604"]


    urunler_df=vericek(carrefour,migros,"Armut")
    if urunler_df is not None : 
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Armut|armut")]
            data=veriekle("Armut",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=ayva%3AbestSeller%3AproductPrimaryCategoryCode%3A1018%3AinStockFlag%3Atrue&text=ayva#"]
    migros=["https://www.migros.com.tr/rest/search/screens/sert-meyveler-c-3ed?sayfa=1&93=606"]


    urunler_df=vericek(carrefour,migros,"Ayva")
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Ayva|ayva")]

            data=veriekle("Ayva",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=%C3%A7ilek%3AbestSeller%3AproductPrimaryCategoryCode%3A1017%3AinStockFlag%3Atrue&text=%C3%A7ilek#"]
    migros=["https://www.migros.com.tr/rest/search/screens/yumusak-meyveler-c-3ee?sayfa=1&93=614"]

    urunler_df=vericek(carrefour,migros,name="Çilek")


    data=veriekle("Çilek",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=elma%3AbestSeller%3AproductPrimaryCategoryCode%3A1018%3AinStockFlag%3Atrue&text=elma#"]
    migros=["https://www.migros.com.tr/rest/search/screens/sert-meyveler-c-3ed?sayfa=1&93=618"]


    urunler_df=vericek(carrefour,migros,"Elma")
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Elma|elma")]


            data=veriekle("Elma",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=karpuz%3Arelevance%3AproductPrimaryCategoryCode%3A1018%3AinStockFlag%3Atrue&text=karpuz#"]
    migros=["https://www.migros.com.tr/rest/search/screens/kavun-ve-karpuz-c-3eb"]

    urunler_df=vericek(carrefour,migros,name="Karpuz")

    if urunler_df is not None: 
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Karpuz|karpuz")]


            data=veriekle("Karpuz",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=kavun%3AbestSeller%3AinStockFlag%3Atrue%3AproductPrimaryCategoryCode%3A1018"]
    migros=["https://www.migros.com.tr/rest/search/screens/kavun-ve-karpuz-c-3eb"]

    urunler_df=vericek(carrefour,migros,"Kavun")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Kavun|kavun", regex=True)]


            data=veriekle("Kavun",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search/?q=kivi%3AbestSeller%3AinStockFlag%3Atrue&text=kivi#"]
    migors=["https://www.migros.com.tr/rest/search/screens/egzotik-meyveler-c-3ea"]


    urunler_df=vericek(carrefour,migros,"Kivi")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("ml")]
            urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("Armut|armut", regex=True)]
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("kivi|Kivi", regex=True)]


            data=veriekle("Kivi",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=limon%3AbestSeller%3AproductPrimaryCategoryCode%3A1016%3AinStockFlag%3Atrue&text=limon#"]
    migros=["https://www.migros.com.tr/rest/search/screens/narenciye-c-3ec?sayfa=1&93=639"]



    urunler_df=vericek(carrefour,migros,"Limon")
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Limon|limon", regex=True)]

            data=veriekle("Limon",data,urunler_df)





    carrefour=["https://www.carrefoursa.com/search/?q=mandalina%3AbestSeller%3AinStockFlag%3Atrue&text=mandalina#"]
    migros=["https://www.migros.com.tr/rest/search/screens/narenciye-c-3ec?sayfa=1&93=640"]



    urunler_df=vericek(carrefour,migros,"Mandalina")
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Mandalina|mandalina", regex=True)]



            data=veriekle("Mandalina",data,urunler_df)




    carrefour=["https://www.carrefoursa.com/search?q=muz%3AbestSeller%3AproductPrimaryCategoryCode%3A1022%3AinStockFlag%3Atrue&text=muz#"]
    migros=["https://www.migros.com.tr/rest/search/screens/egzotik-meyveler-c-3ea?sayfa=1&93=645"]


    urunler_df=vericek(carrefour,migros,"Muz")
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Muz|muz", regex=True)]
            data=veriekle("Muz",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=%C5%9Feftali%3AbestSeller%3AproductPrimaryCategoryCode%3A1018%3AinStockFlag%3Atrue&text=%C5%9Feftali#"]
    migros=["https://www.migros.com.tr/rest/search/screens/yumusak-meyveler-c-3ee"]

    urunler_df=vericek(carrefour,migros,"Şeftali")



    data=veriekle("Şeftali",data,urunler_df)
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Şeftali|şeftali", regex=True)]


            data=veriekle("Şeftali",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=nar%3AbestSeller%3AproductPrimaryCategoryCode%3A1018%3AinStockFlag%3Atrue&text=nar#"]
    migros=["https://www.migros.com.tr/rest/search/screens/sert-meyveler-c-3ed?sayfa=1&93=646"]



    urunler_df=vericek(carrefour,migros,"Nar")
    if urunler_df is not None:
        if not urunler_df.empty:

            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Nar Kg|Nar kg|nar kg|nar Kg|Nar|nar", regex=True)]


            data=veriekle("Nar",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=badem%3AbestSeller%3AproductPrimaryCategoryCode%3A1519%3AinStockFlag%3Atrue&text=badem#"]
    migros=["https://www.migros.com.tr/rest/search/screens/badem-c-280f?sayfa=1&markalar=6570,425,4373,7923,652"]



    urunler_df=vericek(carrefour,migros,"Badem İçi")
    if urunler_df is not None:
        if not urunler_df.empty:
            
            data=veriekle("Badem İçi",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=ceviz%3AbestSeller%3AproductPrimaryCategoryCode%3A1519%3AinStockFlag%3Atrue&text=ceviz#"]
    migros=["https://www.migros.com.tr/rest/search/screens/ceviz-ici-c-2805"]



    urunler_df=vericek(carrefour,migros,"Ceviz İçi")
    if urunler_df is not None:
        if not urunler_df.empty:
            data=veriekle("Ceviz İçi",data,urunler_df)




    carrefour=["https://www.carrefoursa.com/search?q=f%C4%B1nd%C4%B1k%3AbestSeller%3AproductPrimaryCategoryCode%3A1519%3AinStockFlag%3Atrue&text=f%C4%B1nd%C4%B1k#"]
    migros=["https://www.migros.com.tr/rest/search/screens/findik-c-280e"]



    urunler_df=vericek(carrefour,migros,"Fındık İçi")
    if urunler_df is not None:
        if not urunler_df.empty:


            data=veriekle("Fındık İçi",data,urunler_df)




    migros=["https://www.migros.com.tr/rest/search/screens/antep-fistigi-c-280d"]


    urunler_df=vericek("",migros,name="Antep Fıstığı")
    if urunler_df is not None:
        if not urunler_df.empty:
            data=veriekle("Antep Fıstığı",data,urunler_df)




    migros=["https://www.migros.com.tr/rest/search/screens/fistik-c-280c"]


    urunler_df=vericek("",migros,name="Yer Fıstığı")
    if urunler_df is not None:
        if not urunler_df.empty:


            data=veriekle("Yer Fıstığı",data,urunler_df)













    carrefour=["https://www.carrefoursa.com/search/?q=leblebi%3AbestSeller%3AinStockFlag%3Atrue&text=leblebi#"]
    migros=["https://www.migros.com.tr/rest/search/screens/leblebi-c-2811"]



    urunler_df=vericek(carrefour,migros,"Leblebi")
    if urunler_df is not None:
        if not urunler_df.empty:
            data=veriekle("Leblebi",data,urunler_df)






    carrefour=["https://www.carrefoursa.com/search/?q=ay%C3%A7ekirde%C4%9Fi%3AbestSeller%3AinStockFlag%3Atrue&text=ay%C3%A7ekirde%C4%9Fi#"]
    migros=["https://www.migros.com.tr/rest/search/screens/aycekirdegi-c-280b"]



    urunler_df=vericek(carrefour,migros,"Ay Çekirdeği")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Ayçekirdeği|Ayçekirdek|Şimşek")]



            data=veriekle("Ay Çekirdeği",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search/?q=kabak+%C3%A7ekirde%C4%9Fi%3AbestSeller%3AinStockFlag%3Atrue&text=kabak+%C3%A7ekirde%C4%9Fi#"]
    migros=["https://www.migros.com.tr/rest/search/screens/kabak-cekirdegi-c-280a"]


    urunler_df=vericek(carrefour,migros,"Kabak Çekirdeği")

    if urunler_df is not None:
        if not urunler_df.empty:
            


            data=veriekle("Kabak Çekirdeği",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=Kuru+%C3%9Cz%C3%BCm%3AbestSeller%3AinStockFlag%3Atrue%3AproductPrimaryCategoryCode%3A1519"]
    migros=["https://www.migros.com.tr/rest/search/screens/kuru-uzum-c-27fd"]



    urunler_df=vericek(carrefour,migros,"Kuru Üzüm")
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Üzüm|üzüm")]


            data=veriekle("Kuru Üzüm",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=kuru+kay%C4%B1s%C4%B1%3AbestSeller%3AproductPrimaryCategoryCode%3A1519%3AinStockFlag%3Atrue&text=kuru+kay%C4%B1s%C4%B1#"]
    migros=["https://www.migros.com.tr/rest/search/screens/kuru-kayisi-c-2800"]


    urunler_df=vericek(carrefour,migros,"Kuru Kayısı")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("kayısı|Kayısı")]



            data=veriekle("Kuru Kayısı",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=%C3%A7arliston%3AbestSeller%3AproductPrimaryCategoryCode%3A1027%3AinStockFlag%3Atrue&text=%C3%A7arliston#"]
    migros=["https://www.migros.com.tr/rest/search/screens/mevsim-sebzeleri-c-3f4?sayfa=1&94=686"]


    urunler_df=vericek(carrefour,migros,"Çarliston Biber")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Çarliston|çarliston")]

            data=veriekle("Çarliston Biber",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=Dolmal%C4%B1k+Biber%3AbestSeller%3AproductPrimaryCategoryCode%3A1027"]
    migros=["https://www.migros.com.tr/rest/search/screens/mevsim-sebzeleri-c-3f4?sayfa=1&94=686"]



    urunler_df=vericek(carrefour,migros,"Dolmalık Biber")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Dolma|Dolmalık")]


            data=veriekle("Dolmalık Biber",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=Sivri+Biber%3AbestSeller%3AproductPrimaryCategoryCode%3A1027%3AinStockFlag%3Atrue&text=Sivri+Biber#"]
    migros=["https://www.migros.com.tr/rest/search/screens/mevsim-sebzeleri-c-3f4?sayfa=1&94=686"]


    urunler_df=vericek(carrefour,migros,"Sivri Biber")

    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("sivri|Sivri")]

            data=veriekle("Sivri Biber",data,urunler_df)




    carrefour=["https://www.carrefoursa.com/search/?q=dereotu%3AbestSeller%3AinStockFlag%3Atrue&text=dereotu#"]
    migros=["https://www.migros.com.tr/rest/search/screens/otlar-yesillikler-c-3f5"]

    urunler_df=vericek(carrefour,migros,"Dereotu")
    if urunler_df is not None :
        if  not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Adet|adet")]


            data=veriekle("Dereotu",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=domates%3AbestSeller%3AinStockFlag%3Atrue%3Acategory%3A1014"]
    migros=["https://www.migros.com.tr/rest/search/screens/mevsim-sebzeleri-c-3f4?sayfa=1&94=700"]



    urunler_df=vericek(carrefour,migros,"Domates")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("Salçası", regex=True)]
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Domates|domates", regex=True)]
        

            data=veriekle("Domates",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=taze+fasulye%3AbestSeller%3AinStockFlag%3Atrue%3AproductPrimaryCategoryCode%3A1031"]
    migros=["https://www.migros.com.tr/rest/search/screens/mevsim-sebzeleri-c-3f4"]



    urunler_df=vericek(carrefour,migros,"Taze Fasulye")
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Fasulye|fasulye")]

            data=veriekle("Taze Fasulye",data,urunler_df)






    carrefour=["https://www.carrefoursa.com/search?q=havu%C3%A7%3AbestSeller%3Acategory%3A1014"]
    migros=["https://www.migros.com.tr/rest/search/screens/mevsim-sebzeleri-c-3f4?sayfa=1&94=684"]


    urunler_df=vericek(carrefour,migros,"Havuç")
    if urunler_df is not None:
        if not urunler_df.empty:
            

            data=veriekle("Havuç",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=%C4%B1spanak%3AbestSeller%3AproductPrimaryCategoryCode%3A1030%3AinStockFlag%3Atrue&text=%C4%B1spanak#"]
    migros=["https://www.migros.com.tr/rest/search/screens/otlar-yesillikler-c-3f5?sayfa=1&94=712"]    
        


    urunler_df=vericek(carrefour,migros,"Ispanak")
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Ispanak|ıspanak")]

            data=veriekle("Ispanak",data,urunler_df)







    carrefour=["https://www.carrefoursa.com/search/?q=kabak%3AbestSeller%3AinStockFlag%3Atrue&text=kabak#"]
    migros=["https://www.migros.com.tr/rest/search/screens/mevsim-sebzeleri-c-3f4?sayfa=1&94=713"]    



    urunler_df=vericek(carrefour,migros,"Kabak")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Sakız|Dolmalık|Adet")]

            data=veriekle("Kabak",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=karnabahar%3AbestSeller%3Acategory%3A1014%3AinStockFlag%3Atrue&text=karnabahar#"]
    migros=["https://www.migros.com.tr/rest/search/screens/mevsim-sebzeleri-c-3f4?sayfa=1&94=668"] 


    urunler_df=vericek(carrefour,migros,"Karnabahar")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Karnabahar")]


            data=veriekle("Karnabahar",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=kuru+so%C4%9Fan%3AbestSeller%3AproductPrimaryCategoryCode%3A1033%3AinStockFlag%3Atrue&text=kuru+so%C4%9Fan#"]
    migros=["https://www.migros.com.tr/rest/search/screens/patates-sogan-sarimsak-c-3f6?sayfa=1&94=674"] 


    urunler_df=vericek(carrefour,migros,"Kuru Soğan")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("soğan|Soğan")]

            data=veriekle("Kuru Soğan",data,urunler_df)



    migros=["https://www.migros.com.tr/rest/search/screens/mevsim-sebzeleri-c-3f4?sayfa=1&94=694"] 
    carrefour=["https://www.carrefoursa.com/salata-malzemeleri-/c/1027"]



    urunler_df=vericek(carrefour,migros,name="Beyaz Lahana")
    if urunler_df is not None:
        if not urunler_df.empty:

            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Beyaz Lahana|beyaz lahana|beyaz lahana|Lahana beyaz|Lahana Beyaz")]
            urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("Hazır|hazır")]


            data=veriekle("Beyaz Lahana",data,urunler_df)



    migros=["https://www.migros.com.tr/rest/search/screens/otlar-yesillikler-c-3f5?sayfa=1&94=723"] 
    carrefour=["https://www.carrefoursa.com/salata-malzemeleri-/c/1027"]


    urunler_df=vericek(carrefour,migros,name="Kırmızı Lahana")
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Kırmızı Lahana|kırmızı lahana|Kırmızı lahana|Lahana Kırmızı|Lahana kırmızı")]
            urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("Hazır|hazır")]

            data=veriekle("Kırmızı Lahana",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search/?q=mantar%3AbestSeller%3AinStockFlag%3Atrue&text=mantar#"]
    migros=["https://www.migros.com.tr/rest/search/screens/egzotik-sebzeler-c-3f2"] 


    urunler_df=vericek(carrefour,migros,"Mantar")


    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Mantar|mantar|mantarı|Mantarı")]
            urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("Çorba|Pano|İstiridye|Salatası")]

            data=veriekle("Mantar",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=k%C4%B1v%C4%B1rc%C4%B1k%3AbestSeller%3AinStockFlag%3Atrue%3Acategory%3A1014"]
    migros=["https://www.migros.com.tr/rest/search/screens/otlar-yesillikler-c-3f5?sayfa=1&94=724"] 



    urunler_df=vericek(carrefour,migros,"Kıvırcık")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("kıvırcık|Kıvırcık")]
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Adet|adet", regex=True)]
            data=veriekle("Kıvırcık",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search/?q=maydonoz%3AbestSeller%3AinStockFlag%3Atrue&text=maydonoz#"]
    migros=["https://www.migros.com.tr/rest/search/screens/otlar-yesillikler-c-3f5?sayfa=1&94=733"] 



    urunler_df=vericek(carrefour,migros,"Maydanoz")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("maydanoz|Maydanoz")]

            data=veriekle("Maydanoz",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=nane%3AbestSeller%3AproductPrimaryCategoryCode%3A1030%3AinStockFlag%3Atrue&text=nane#"]
    migros=["https://www.migros.com.tr/rest/search/screens/otlar-yesillikler-c-3f5?sayfa=1&94=736"] 



    urunler_df=vericek(carrefour,migros,"Nane")

    if urunler_df is not None :
        if not urunler_df.empty:
            

            data=veriekle("Nane",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=patl%C4%B1can%3AbestSeller%3AproductPrimaryCategoryCode%3A1031%3AinStockFlag%3Atrue&text=patl%C4%B1can#"]
    migros=["https://www.migros.com.tr/rest/search/screens/mevsim-sebzeleri-c-3f4?sayfa=1&94=727"] 


    urunler_df=vericek(carrefour,migros,"Patlıcan")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("Minyatür")]

            data=veriekle("Patlıcan",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search/?q=p%C4%B1rasa%3AbestSeller%3AinStockFlag%3Atrue&text=p%C4%B1rasa#"]
    migros=["https://www.migros.com.tr/rest/search/screens/otlar-yesillikler-c-3f5?sayfa=1&94=667"] 



    urunler_df=vericek(carrefour,migros,"Pırasa")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("pırasa|Pırasa")]

            data=veriekle("Pırasa",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search/?q=roka%3AbestSeller%3AinStockFlag%3Atrue&text=roka#"]
    migros=["https://www.migros.com.tr/rest/search/screens/otlar-yesillikler-c-3f5?sayfa=1&94=745"] 



    urunler_df=vericek(carrefour,migros,"Roka")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("roka|Roka")]
            data=veriekle("Roka",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=salatal%C4%B1k%3AbestSeller%3Acategory%3A1014%3AinStockFlag%3Atrue&text=salatal%C4%B1k#"]
    migros=["https://www.migros.com.tr/rest/search/screens/mevsim-sebzeleri-c-3f4?sayfa=1&94=708"] 


    urunler_df=vericek(carrefour,migros,"Salatalık")
    if urunler_df is not None :
        if not urunler_df.empty:
            

            data=veriekle("Salatalık",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/kuru-sarimsak-kg-p-30024962"]
    migros=["https://www.migros.com.tr/rest/search/screens/patates-sogan-sarimsak-c-3f6?sayfa=1&94=671"] 


    urunler_df=vericek(carrefour,migros,"Sarımsak")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Sarımsak", regex=True)]

            data=veriekle("Sarımsak",data,urunler_df)






    migros=["https://www.migros.com.tr/rest/search/screens/otlar-yesillikler-c-3f5?sayfa=1&94=740"] 


    urunler_df=vericek("",migros,name="Kırmızı Turp")
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Kırmızı|kırmızı")]
            data=veriekle("Kırmızı Turp",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=patates%3AbestSeller%3AproductPrimaryCategoryCode%3A1033%3AinStockFlag%3Atrue&text=patates#"]
    migros=["https://www.migros.com.tr/rest/search/screens/patates-sogan-sarimsak-c-3f6?sayfa=1&94=678"] 



    urunler_df=vericek(carrefour,migros,"Patates")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("Sarımsak|Soğan", regex=True)]
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("patates|Patates", regex=True)]
            data=veriekle("Patates",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=kuru+fasulye%3AbestSeller%3AinStockFlag%3Atrue%3Acategory%3A1110"]
    migros=["https://www.migros.com.tr/rest/search/screens/fasulye-c-2794"] 



    urunler_df=vericek(carrefour,migros,"Kuru Fasulye")
    if urunler_df is not None : 
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("fasulye|Fasulye")]

            data=veriekle("Kuru Fasulye",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=nohut%3AbestSeller%3AproductPrimaryCategoryCode%3A1152%3AinStockFlag%3Atrue&text=nohut#"]
    migros=["https://www.migros.com.tr/rest/search/screens/nohut-c-2798"]


    urunler_df=vericek(carrefour,migros,"Nohut")
    data=veriekle("Nohut",data,urunler_df)
    if urunler_df is not None:
        if not urunler_df.empty:
            

            data=veriekle("Nohut",data,urunler_df)



    migros=["https://www.migros.com.tr/rest/search/screens/mercimek-c-113c9"]



    urunler_df=vericek("",migros,"Mercimek")
    if urunler_df is not None:
        if not urunler_df.empty:
            data=veriekle("Mercimek",data,urunler_df)





    carrefour=["https://www.carrefoursa.com/diger-sebze/c/1193?q=%3AbestSeller&show=All","https://www.carrefoursa.com/yesil-sebze/c/1187?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]
    migros=["https://www.migros.com.tr/rest/search/screens/konserve-sebze-c-2852"]


    urunler_df=vericek(carrefour,migros,"Konserveler")
    if urunler_df is not None:

        if not urunler_df.empty:
            
            data=veriekle("Konserveler",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=tur%C5%9Fu%3AbestSeller%3AinStockFlag%3Atrue&text=tur%C5%9Fu#"]
    migros=["https://www.migros.com.tr/rest/search/screens/tursu-c-454"]


    urunler_df=vericek(carrefour,migros,"Turşu")
    if urunler_df is not None:
        if not urunler_df.empty:
            data=veriekle("Turşu",data,urunler_df)




    carrefour=["https://www.carrefoursa.com/salca/c/1180?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]
    migros=["https://www.migros.com.tr/rest/search/screens/salca-c-453"]



    urunler_df=vericek(carrefour,migros,"Salça")
    if urunler_df is not None:
        if not urunler_df.empty:
            data=veriekle("Salça",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/zeytin/c/1356?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]
    migros=["https://www.migros.com.tr/rest/search/screens/zeytin-c-71"]



    urunler_df=vericek(carrefour,migros,"Zeytin")
    if urunler_df is not None:
        if not urunler_df.empty:
            data=veriekle("Zeytin",data,urunler_df)





    carrefour=["https://www.carrefoursa.com/cipsler/c/1552?q=%3AbestSeller%3Acategory%3A1552%3AinStockFlag%3Atrue&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/patates-cipsi-c-27f9"]


    urunler_df=vericek(carrefour,migros,"Cipsler")
    if urunler_df is not None:
        if not urunler_df.empty:
            data=veriekle("Cipsler",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search/?q=Toz+%C5%9Eeker%3AbestSeller%3AinStockFlag%3Atrue&text=Toz+%C5%9Eeker#"]
    migros=["https://www.migros.com.tr/rest/search/screens/toz-seker-c-544"
    ]



    urunler_df=vericek(carrefour,migros,"Toz Şeker")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Toz", regex=True)]
            data=veriekle("Toz Şeker",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search/?q=k%C3%BCp+%C5%9Feker%3AbestSeller%3AinStockFlag%3Atrue&text=k%C3%BCp+%C5%9Feker#"]
    migros=["https://www.migros.com.tr/rest/search/screens/kup-seker-c-543"
    ]


    urunler_df=vericek(carrefour,migros,"Kesme Şeker")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Küp|kesme|Kesme", regex=True)]
            data=veriekle("Kesme Şeker",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=re%C3%A7el%3AbestSeller%3AinStockFlag%3Atrue&text=re%C3%A7el#"]
    migros=["https://www.migros.com.tr/rest/search/screens/recel-c-277b"
    ]


    urunler_df=vericek(carrefour,migros,"Reçel")
    if urunler_df is not None:
        if not urunler_df.empty:
            data=veriekle("Reçel",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/ballar/c/1362?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/cam-bali-c-2768"
    ]


    urunler_df=vericek(carrefour,migros,"Bal")
    if urunler_df is not None:
        if not urunler_df.empty:
            data=veriekle("Bal",data,urunler_df)




    carrefour=["https://www.carrefoursa.com/search?q=pekmez%3AbestSeller&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/pekmez-c-2770"
    ]


    urunler_df=vericek(carrefour,migros,"Pekmez")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Pekmez|Pekmezi", regex=True)]
            data=veriekle("Pekmez",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=tahin+helva%3AbestSeller%3AproductPrimaryCategoryCode%3A1374%3AinStockFlag%3Atrue&text=tahin+helva#"]
    migros=["https://www.migros.com.tr/rest/search/screens/helva-c-2771?sayfa=1&markalar=425"
    ]


    urunler_df=vericek(carrefour,migros,"Tahin Helvası")

    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Helvası|Helva", regex=True)]
            data=veriekle("Tahin Helvası",data,urunler_df)




    carrefour=["https://www.carrefoursa.com/search?q=f%C4%B1nd%C4%B1k+ezmesi%3AbestSeller%3AinStockFlag%3Atrue&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/findik-ezmesi-c-2778"
    ]


    urunler_df=vericek(carrefour,migros,"Fındık Ezmesi")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Fındık Ezmesi", regex=True)]
            data=veriekle("Fındık Ezmesi",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=tablet+%C3%A7ikolata%3AbestSeller%3AinStockFlag%3Atrue&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/cikolata-c-439"
    ]


    urunler_df=vericek(carrefour,migros,"Çikolata Tablet")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Tablet|Kare", regex=True)]
            data=veriekle("Çikolata Tablet",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=krem+%C3%A7ikolata%3AbestSeller%3AproductPrimaryCategoryCode%3A1381%3AinStockFlag%3Atrue&text=krem+%C3%A7ikolata#"]
    migros=["https://www.migros.com.tr/rest/search/screens/kakao-findik-kremalari-c-2779"
    ]


    urunler_df=vericek(carrefour,migros,"Çikolata Krem")

    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("Ezmesi", regex=True)]
            data=veriekle("Çikolata Krem",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=lokum%3AbestSeller%3AproductPrimaryCategoryCode%3A1494%3AinStockFlag%3Atrue&text=lokum#"]
    migros=["https://www.migros.com.tr/rest/search/screens/lokum-c-281c"
    ]

    urunler_df=vericek(carrefour,migros,"Lokum")
    if urunler_df is not None :
        if not urunler_df.empty:
            data=veriekle("Lokum",data,urunler_df)






    carrefour=["https://www.carrefoursa.com/search?q=sak%C4%B1z%3Arelevance%3AinStockFlag%3Atrue%3AproductPrimaryCategoryCode%3A1503","https://www.carrefoursa.com/search?q=sak%C4%B1z%3Arelevance%3AproductPrimaryCategoryCode%3A1502%3AinStockFlag%3Atrue&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/sakiz-c-443"
    ]


    urunler_df=vericek(carrefour,migros,"Sakız")
    if urunler_df is not None :
        if not urunler_df.empty:
            data=veriekle("Sakız",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/sekerleme/c/1494?q=%3AbestSeller%3Acategory%3ABRN-1949%3Acategory%3ABRN-2504%3Acategory%3ABRN-3185%3Acategory%3ABRN-2125%3Acategory%3ABRN-3091%3Acategory%3ABRN-2999%3AinStockFlag%3Atrue&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/geleneksel-sekerleme-c-2819"
    ]


    urunler_df=vericek(carrefour,migros,"Kağıtlı Şeker")
    if urunler_df is not None :
        if not urunler_df.empty:
            data=veriekle("Kağıtlı Şeker",data,urunler_df)




    carrefour=["https://www.carrefoursa.com/kap-dondurma/c/1261?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All",
            "https://www.carrefoursa.com/tek-dondurma/c/1266?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/dondurma-c-41b"
    ]


    urunler_df=vericek(carrefour,migros,"Dondurma")

    data=veriekle("Dondurma",data,urunler_df)




    carrefour=["https://www.carrefoursa.com/baharat/c/1167?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/baharat-c-435"
    ]



    urunler_df=vericek(carrefour,migros,"Baharat")
    data=veriekle("Baharat",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/tuz/c/1166?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]
    migros=["https://www.migros.com.tr/rest/search/screens/tuz-c-436"
    ]


    urunler_df=vericek(carrefour,migros,"Tuz")
    data=veriekle("Tuz",data,urunler_df)





    carrefour=["https://www.carrefoursa.com/search?q=kabartma%3AbestSeller%3AproductPrimaryCategoryCode%3A1282%3AinStockFlag%3Atrue&text=kabartma#"]
    migros=["https://www.migros.com.tr/rest/search/screens/kabartma-tozu-sekerli-vanilin-c-2893"
    ]



    urunler_df=vericek(carrefour,migros,"Kabartma Maddeleri")
    data=veriekle("Kabartma Maddeleri",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search?q=sirke%3AbestSeller%3AproductPrimaryCategoryCode%3A1219%3AinStockFlag%3Atrue&text=sirke#"]
    migros=["https://www.migros.com.tr/rest/search/screens/sirke-c-284f"
    ]


    urunler_df=vericek(carrefour,migros,"Sirke")
    data=veriekle("Sirke",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/search/?q=ket%C3%A7ap%3AbestSeller%3AinStockFlag%3Atrue&text=ket%C3%A7ap#"]
    migros=["https://www.migros.com.tr/rest/search/screens/ketcap-c-2847"
    ]


    urunler_df=vericek(carrefour,migros,"Ketçap")
    if urunler_df is not None and not urunler_df.empty:
        urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("Mayonez", regex=True)]
        urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Ketçap")]

        data=veriekle("Ketçap",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/search?q=mayonez%3AbestSeller%3AproductPrimaryCategoryCode%3A1212%3AinStockFlag%3Atrue&text=mayonez#"]
    migros=["https://www.migros.com.tr/rest/search/screens/mayonez-c-2848"
    ]


    urunler_df=vericek(carrefour,migros,"Mayonez")
    data=veriekle("Mayonez",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/tahin-pekmez-helva/c/1374?q=%3AbestSeller%3Acategory%3A1310%3AinStockFlag%3Atrue&text=#"]
    migros=["https://www.migros.com.tr/rest/search/screens/tahin-c-276f"
    ]



    urunler_df=vericek(carrefour,migros,"Tahin")
    if urunler_df is not None and not urunler_df.empty:
        urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Tahin", regex=True)]
        urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("Pekmezi|Helva", regex=True)]
        data=veriekle("Tahin",data,urunler_df)




    carrefour=["https://www.carrefoursa.com/hazir-corbalar/c/1224?q=%3AbestSeller&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/pratik-yemek-c-44f?sayfa=1&markalar=424,483"
    ]


    urunler_df=vericek(carrefour,migros,"Hazır Çorbalar")
    if urunler_df is not None:
        if not urunler_df.empty:
            urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Çorba|çorba|çorbası|Çorbası", regex=True)]
            data=veriekle("Hazır Çorbalar",data,urunler_df)





    carrefour=["https://www.carrefoursa.com/hazirlanacak-tatlilar/c/1300?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/puding-c-41d"
    ]


    urunler_df=vericek(carrefour,migros,"Hazır Pakette Toz Tatlılar (Puding)")
    urunler_df=urunler_df[urunler_df["Ürün"].str.contains("puding|Puding|pudingi|Pudingi", regex=True)]
    data=veriekle("Hazır Pakette Toz Tatlılar (Puding)",data,urunler_df)





    carrefour=["https://www.carrefoursa.com/search/?q=t%C3%BCrk+kahvesi%3AbestSeller%3AinStockFlag%3Atrue&text=t%C3%BCrk+kahvesi#"]
    migros=["https://www.migros.com.tr/rest/search/screens/filtre-kahve-c-11223","https://www.migros.com.tr/rest/search/screens/turk-kahvesi-c-28c4","https://www.migros.com.tr/rest/search/screens/kapsul-kahve-c-11224","https://www.migros.com.tr/rest/search/screens/cekirdek-kahve-c-11225"
    ]


    urunler_df=vericek(carrefour,migros,"Kahve")
    data=veriekle("Kahve",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/kahve/c/1467?q=%3AbestSeller%3Acategory%3A1467%3AinStockFlag%3Atrue&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/hazir-kahve-c-11222"
    ]



    urunler_df=vericek(carrefour,migros,"Hazır Kahve")
    if urunler_df is not None :
        if not urunler_df.empty:
            urunler_df=urunler_df[~urunler_df["Ürün"].str.contains("Türk", regex=True)]
            data=veriekle("Hazır Kahve",data,urunler_df)






    migros=["https://www.migros.com.tr/rest/search/screens/dokme-cay-c-28c1"
    ]


    urunler_df=vericek("",migros,name="Çay")
    data=veriekle("Çay",data,urunler_df)



    migros=["https://www.migros.com.tr/rest/search/screens/bitki-cayi-c-28c0"
    ]

    urunler_df=vericek("",migros,name="Bitki ve Meyve Çayı (Poşet)")
    data=veriekle("Bitki ve Meyve Çayı (Poşet)",data,urunler_df)





    migros=["https://www.migros.com.tr/rest/search/screens/uzun-omurlu-sut-c-40a?sayfa=1&109=1005"
    ]


    urunler_df=vericek("",migros,name="Kakaolu Toz İçecekler")
    data=veriekle("Kakaolu Toz İçecekler",data,urunler_df)



    carrefour=["https://www.carrefoursa.com/sular/c/1411?q=%3AbestSeller%3AinStockFlag%3Atrue&show=All"]

    migros=["https://www.migros.com.tr/rest/search/screens/su-c-84"
    ]

    urunler_df=vericek(carrefour,migros,"Su")
    urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Su|su")]
    data=veriekle("Su",data,urunler_df)





    carrefour=["https://www.carrefoursa.com/maden-sulari/c/1412?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]
    migros=["https://www.migros.com.tr/rest/search/screens/maden-suyu-c-85"
    ]


    urunler_df=vericek(carrefour,migros,"Maden Suyu ve Sodası")
    data=veriekle("Maden Suyu ve Sodası",data,urunler_df)








    carrefour=["https://www.carrefoursa.com/search?q=gazoz%3AbestSeller%3AinStockFlag%3Atrue&text=gazoz#"]
    migros=["https://www.migros.com.tr/rest/search/screens/gazoz-c-467"
    ]


    urunler_df=vericek(carrefour,migros,"Gazoz Meyveli")
    data=veriekle("Gazoz Meyveli",data,urunler_df)










    carrefour=["https://www.carrefoursa.com/kola/c/1419?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]
    migros=["https://www.migros.com.tr/rest/search/screens/kola-c-465"
    ]



    urunler_df=vericek(carrefour,migros,"Kola")
    data=veriekle("Kola",data,urunler_df)





    carrefour=["https://www.carrefoursa.com/search?q=so%C4%9Fuk+%C3%A7ay%3AbestSeller%3AproductPrimaryCategoryCode%3A1450%3AinStockFlag%3Atrue&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/soguk-cay-c-28be"]



    urunler_df=vericek(carrefour,migros,"Soğuk Çay")
    data=veriekle("Soğuk Çay",data,urunler_df)








    carrefour=["https://www.carrefoursa.com/search?q=ayran%3AbestSeller&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/ayran-c-47a"
            
    ]


    urunler_df=vericek(carrefour,migros,"Ayran")
    urunler_df=urunler_df[urunler_df["Ürün"].str.contains("Ayran|ayran", regex=True)]

    data=veriekle("Ayran",data,urunler_df)




    carrefour=["https://www.carrefoursa.com/search?q=meyve+suyu%3AbestSeller%3AinStockFlag%3Atrue&show=All"]
    migros=["https://www.migros.com.tr/rest/search/screens/meyve-suyu-c-46c"
            
    ]


    urunler_df=vericek(carrefour,migros,"Meyve Suyu")
    data=veriekle("Meyve Suyu",data,urunler_df)








    migros=["https://www.migros.com.tr/rest/search/screens/tulum-peyniri-c-2734"
            
    ]


    urunler_df=vericek("",migros,name="Tulum Peyniri")
    data=veriekle("Tulum Peyniri",data,urunler_df)


    carrefour=["https://www.carrefoursa.com/kakao/c/1304?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#"]
    migros=["https://www.migros.com.tr/rest/search/screens/kakao-c-2894"
            
    ]


    urunler_df=vericek(carrefour,migros,name="Kakao")
    data=veriekle("Kakao",data,urunler_df)


    for col in data.columns[1:]:
        data[col]=data[col].astype(float)
                    



    df_filled_corrected=data.copy()
                    




    df_filled_corrected.to_csv("sepet.csv")


    ağırlıklar=pd.read_csv("ağırlıklar.csv")
    ağırlıklar=ağırlıklar.set_index(ağırlıklar["Ürün"])
    ağırlıklar=ağırlıklar.drop("Ürün",axis=1)
    ağırlıklar=ağırlıklar.sort_index()



    gfe=pd.read_csv("gfe.csv")
    gfe=gfe.set_index(pd.to_datetime(gfe["Tarih"]))
    gfe=gfe.drop("Tarih",axis=1)



    data1=df_filled_corrected.copy().sort_index()
    data1=data1.drop_duplicates()
        
    degisim=(((data1.iloc[:,-1]/data1.iloc[:,-2])-1)*100)

    grup_degisim=(((data1.iloc[:,-1]/data1.iloc[:,-2])-1)*100).dropna()
    grup_degisim=(((grup_degisim/100)+1).groupby(level=0).apply(lambda x: np.prod(x)**(1/len(x))).sort_index()-1)*100


    degisim=degisim.fillna(grup_degisim)
    degisim = ((((degisim/100)+1).groupby(level=0).apply(lambda x: np.prod(x)**(1 / len(x))).sort_index())-1)*100

    degisim=degisim.fillna(0)




    ağırlıklar["Değişim"]=degisim



    ağırlıklar[f"Endeks_{bugün}"]=ağırlıklar[f"Endeks_{dün}"]*(1+(ağırlıklar["Değişim"]/100))

    ağırlıklar[f"Ağırlıklı Endeks_{bugün}"]=ağırlıklar[f"Endeks_{bugün}"]*ağırlıklar["Ağırlık"]
    gfe.loc[pd.to_datetime(bugün)]=ağırlıklar[f"Ağırlıklı Endeks_{bugün}"].sum()
    gfe.to_csv("gfe.csv",index=True)

    csv_file = "gfe_ohlc.csv"
    df = pd.read_csv(csv_file, index_col=0)

    df.index=pd.to_datetime(df.index)
    today_date = datetime.today().strftime("%Y-%m-%d %H:%M:00")



    try:
        df.loc[pd.to_datetime(today_date)] = ağırlıklar[f"Ağırlıklı Endeks_{bugün}"].sum()
    except:
        pass

    df.to_csv(csv_file)

    df=df.resample('D').agg(
        high=("GFE", "max"),         
        low=("GFE", "min"),         
        open=("GFE", "first"),   
        close=("GFE", "last")  )
    ohlc=pd.read_csv("ohlc.csv").set_index("Unnamed: 0")
    ohlc.index=pd.to_datetime(ohlc.index)
    ohlc.columns=["high","low","open","close"]
    ohlc=pd.concat([ohlc,df],axis=0)
    ohlc = ohlc[~ohlc.index.duplicated(keep='last')]
    ohlc.to_csv("ohlc.csv")


    endeks_sutunlari = ağırlıklar.filter(like='Endeks_')
    endeksler = [col for col in ağırlıklar.columns if col.startswith('Endeks_')]
    ağırlıklar[endeksler].to_csv("endeksler.csv",index=True)


    ağırlıklar.to_csv("ağırlıklar.csv",index=True)




    tarih=datetime.now().strftime("%Y-%m-%d %H:%M")
    tarih=pd.DataFrame({"Current DateTime": [tarih]})
    tarih.to_csv("tarih.csv")







    endeksler=pd.read_csv("endeksler.csv")
    endeksler=endeksler.set_index(endeksler["Ürün"])
    endeksler=endeksler.drop("Ürün",axis=1)
    endeksler=endeksler.T
    endeksler=endeksler.set_index(pd.date_range(start="2024-10-11",freq="D",periods=len(endeksler)))
    ağırlıklar=pd.read_excel("Weights_2022.xlsx")
    cols=ağırlıklar["Unnamed: 1"].dropna().iloc[2:130].values
    ağırlıklar=ağırlıklar[["Unnamed: 5","Unnamed: 4"]]
    ağırlıklar["Unnamed: 4"]=ağırlıklar["Unnamed: 4"]*100
    ağırlıklar=ağırlıklar.iloc[4:132]
    ağırlıklar=ağırlıklar.fillna(method="ffill")
    ağırlıklar.columns=["Grup","Ağırlık"]
    endeksler=endeksler[cols]
    gruplar=pd.concat([ağırlıklar.reset_index().drop("index",axis=1),endeksler.T.reset_index().iloc[:,1:]],axis=1)
    weighted_sums = gruplar.groupby('Grup').apply(lambda group: group.iloc[:, 2:].mul(group['Ağırlık'], axis=0).sum()).reset_index()

    # Rename columns for clarity
    weighted_sums.columns = ['Grup'] + [f'{col}_Toplam' for col in gruplar.columns[2:]]
    # Calculate total weight for each group
    total_weights = gruplar.groupby('Grup')['Ağırlık'].sum().reset_index()
    total_weights.columns = ['Grup', 'Toplam_Ağırlık']

    # Merge total weights with weighted sums
    weighted_sums = pd.merge(weighted_sums, total_weights, on='Grup')

    # Calculate indices for each date by dividing weighted sum by total weight
    for col in weighted_sums.columns[1:-1]:  # Exclude 'Grup' and 'Toplam_Ağırlık'
        weighted_sums[col] = weighted_sums[col] / weighted_sums['Toplam_Ağırlık']

    # Drop 'Toplam_Ağırlık' for display purposes
    weighted_indices = weighted_sums.drop(columns=['Toplam_Ağırlık'])

    weighted_indices=weighted_indices.T
    cols=weighted_indices.iloc[0,:]
    weighted_indices.columns=cols
    weighted_indices=weighted_indices.iloc[1:,:]
    weighted_indices=weighted_indices.set_index(pd.date_range(start="2024-10-11",freq="D",periods=len(weighted_indices)))


    ağırlıklar=pd.read_excel("Weights_2022.xlsx")
    cols=ağırlıklar["Unnamed: 1"].dropna().iloc[2:130].values
    ağırlıklar=ağırlıklar[["Unnamed: 5","Unnamed: 4"]]
    ağırlıklar["Unnamed: 4"]=ağırlıklar["Unnamed: 4"]*100
    ağırlıklar=ağırlıklar.iloc[4:132]
    ağırlıklar=ağırlıklar.fillna(method="ffill")
    ağırlıklar.columns=["Grup","Ağırlık"]
    endeksler=endeksler[cols]
    gruplar=pd.concat([ağırlıklar.reset_index().drop("index",axis=1),endeksler.T.reset_index().iloc[:,1:]],axis=1)
    weighted_sums = gruplar.groupby('Grup').apply(lambda group: group.iloc[:, 2:].mul(group['Ağırlık'], axis=0).sum()).reset_index()

    # Rename columns for clarity
    weighted_sums.columns = ['Grup'] + [f'{col}_Toplam' for col in gruplar.columns[2:]]
    # Calculate total weight for each group
    total_weights = gruplar.groupby('Grup')['Ağırlık'].sum().reset_index()
    total_weights.columns = ['Grup', 'Toplam_Ağırlık']

    # Merge total weights with weighted sums
    weighted_sums = pd.merge(weighted_sums, total_weights, on='Grup')

    # Calculate indices for each date by dividing weighted sum by total weight
    for col in weighted_sums.columns[1:-1]:  # Exclude 'Grup' and 'Toplam_Ağırlık'
        weighted_sums[col] = weighted_sums[col] / weighted_sums['Toplam_Ağırlık']

    # Drop 'Toplam_Ağırlık' for display purposes
    weighted_indices = weighted_sums.drop(columns=['Toplam_Ağırlık'])

    weighted_indices=weighted_indices.T
    cols=weighted_indices.iloc[0,:]
    weighted_indices.columns=cols
    weighted_indices=weighted_indices.iloc[1:,:]
    weighted_indices=weighted_indices.set_index(pd.date_range(start="2024-10-11",freq="D",periods=len(weighted_indices)))
    weighted_indices=weighted_indices.to_csv("weighted_indices.csv")


    import os
    import subprocess
    from datetime import datetime
    import time
    import git
    from git import Repo
    import os
    repo_dir = ".git"  # Buraya Git deposunun yolunu girin

    def git_add_commit_push():
        try:
            # Repo nesnesini oluştur
            repo = Repo(repo_dir)
            assert not repo.bare

            # Git add: tüm değişiklikleri ekliyoruz
            repo.git.add(A=True)  # A=True ile tüm dosyalar eklenir

            # Commit işlemi
            commit_message = "update"
            repo.index.commit(commit_message)
            print(f"Commit işlemi başarılı: {commit_message}")

            # Push işlemi
            origin = repo.remote(name='origin')
            origin.push()
            print("Push işlemi başarılı.")

        except Exception as e:
            print(f"Git işlemi sırasında hata oluştu: {e}")

    time.sleep(1200)
    git_add_commit_push()


        




Döngü başladı, saat: 16:41:36
Scraping URL: https://www.carrefoursa.com/pirinc/c/1134?q=%3AbestSeller%3AinStockFlag%3Atrue&text=#
